# Task3. 酶的EC号预测

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-06-03  

## 任务简介
该任务通过给定酶序列，预测该酶的反应类别（EC号）


## 数据统计
- 遵照任务1找到的时间节点对数据进行划分
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 


|     Items    | 单功能酶       |   多功能非酶    |合计                     |
| ------------ | --------| --------- |----------------------------------|
| 训练集        | 185,453 | 13,239    | 198,692（198,692/219,227=90.63%) |
| 测试集        | 18,868  | 25,594    | 20,535 (20,535/219,227=9.37% )   |


## 数据集构建方法

* 根据蛋白注释信息与之前划定的酶与非酶数据集，将「酶」数据进行分类。
* 有1个EC号的被定义为「单功能酶」，有多个EC号的被定义为「多功能酶」。


## 实验结果

### 同源比对

|Methods   | Accuracy                        |             Precision           |           Recall               |F1   |
| ---------| ------------------------------- | ------------------------------- |--------------------------------|-----|
| 同源比对  |  0.6345134619461522(11972/18868) | 0.7470360663921128(11972/16026) |0.84937460250159(16026/18868)   |      |




### 机器学习 + onehot


|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix					|
| ------| ----------|-----------| ---------- | ----------|-----------|-----------|---------- |------------------------------------------|





### 机器学习 + unirep

|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix					    |
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|



## 1. 导入必要的包

In [17]:
import numpy as np
import pandas as pd
import random
import sys
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import funclib

from xgboost.sklearn import XGBClassifier


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. 加载数据

In [3]:
train = pd.read_hdf('./data/train.h5',key='data')
test = pd.read_hdf('./data/test.h5',key='data')
head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
head = head + ['ec_label','ec_appears']
train.columns = head
test.columns = head

## 3. 同源比对

In [43]:
res_data=funclib.getblast(train,test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [51]:
# 匹配查询结果
id_map_ec = train[['id', 'ec_number']].append(test[['id', 'ec_number']],ignore_index=True)
id_ec_dict = {v: k for v,k in zip( id_map_ec.id, id_map_ec.ec_number)} 
res_data['is_ec_match']=res_data.apply(lambda x: (id_ec_dict.get(x['id'])== id_ec_dict.get(x['sseqid'])), axis=1)

# 输出比对结果
funclib.evaluateBlast(res_data,train,test, 'ec_number')

Total query records are: 18868
Matched records are: 16026
Accuracy: 0.6345134619461522(11972/18868)
Pricision: 0.7470360663921128(11972/16026)
Recall: 0.84937460250159(16026/18868)


## 4. 机器学习EC号预测
### 4.1 onehot + 机器学习

In [4]:
trainset = train[['id', 'ec_label','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'ec_label','seq', 'seqlength']].reset_index(drop=True)

MAX_SEQ_LENGTH = 1000 #定义序列最长的长度
trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))

f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

train_full = pd.concat([trainset, f_train], axis=1, join='inner' ) #拼合训练集
test_full = pd.concat([testset, f_test], axis=1, join='inner' )    #拼合测试集

X_train = np.array(train_full.iloc[:,4:])
X_test = np.array(test_full.iloc[:,4:])
Y_train = np.array(train_full.ec_label.astype('int'))
Y_test = np.array(test_full.ec_label.astype('int'))



In [7]:
xgboost_clf = XGBClassifier(min_child_weight=6,max_depth=15, objective='multi:softmax',num_class=5, n_jobs=-2)
print("-" * 60)
print("xgboost模型：", xgboost_clf)
clf = xgboost_clf.fit(X_train, Y_train)
res = clf.predict(X_test)
aa = pd.DataFrame()
aa['ground_truth'] = Y_test
aa['pred'] = res
aa['iscorrect']= aa.apply(lambda x: x.ground_truth == x.pred, axis=1)
aa.iscorrect.sum()

------------------------------------------------------------
xgboost模型： XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=15,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-2, num_class=5, num_parallel_tree=None,
              objective='multi:softmax', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)
[18:33:00] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if 

In [16]:
train_sub= train[train.ec_appears>=100]
test_sub= test[test.ec_appears>=100]

In [19]:
trainset = train_sub[['id', 'ec_label','seq', 'seqlength']].reset_index(drop=True)
testset = test_sub[['id', 'ec_label','seq', 'seqlength']].reset_index(drop=True)

MAX_SEQ_LENGTH = 1000 #定义序列最长的长度
trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))

f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

train_full = pd.concat([trainset, f_train], axis=1, join='inner' ) #拼合训练集
test_full = pd.concat([testset, f_test], axis=1, join='inner' )    #拼合测试集

X_train = np.array(train_full.iloc[:,4:])
X_test = np.array(test_full.iloc[:,4:])
Y_train = np.array(train_full.ec_label.astype('int'))
Y_test = np.array(test_full.ec_label.astype('int'))

In [20]:
xgboost_clf = XGBClassifier(min_child_weight=6,max_depth=15, objective='multi:softmax',num_class=5, n_jobs=-2)
print("-" * 60)
print("xgboost模型：", xgboost_clf)
clf = xgboost_clf.fit(X_train, Y_train)
res = clf.predict(X_test)
aa = pd.DataFrame()
aa['ground_truth'] = Y_test
aa['pred'] = res
aa['iscorrect']= aa.apply(lambda x: x.ground_truth == x.pred, axis=1)
aa.iscorrect.sum()

------------------------------------------------------------
xgboost模型： XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=15,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-2, num_class=5, num_parallel_tree=None,
              objective='multi:softmax', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)


/home/shizhenkun/anaconda3/envs/py38/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:37:02] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


1604

In [22]:
1604/len(test_sub)

0.16916262391900444

In [23]:
len(test_sub)

9482

In [6]:
dtc = tree.DecisionTreeClassifier(criterion="entropy")
clf = dtc.fit(X_train, Y_train)
res = clf.predict(X_test)
# print(y_test)
# dot_data = tree.export_graphviz(clf, out_file=None)

1629

In [7]:
iris = load_iris()
x = iris['data']
y = iris['target']
dtc = tree.DecisionTreeClassifier(criterion="entropy")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
clf = dtc.fit(x_train, y_train)
print(clf.predict(x_test))
print(y_test)
dot_data = tree.export_graphviz(clf, out_file=None)

NameError: name 'load_iris' is not defined

In [21]:

160/18000

0.008888888888888889